In [1]:
'''
A Convolutional Network implementation example using TensorFlow library.
This example is using the MNIST database of handwritten digits
(http://yann.lecun.com/exdb/mnist/)

Author: Aymeric Damien
Project: https://github.com/aymericdamien/TensorFlow-Examples/
'''

'\nA Convolutional Network implementation example using TensorFlow library.\nThis example is using the MNIST database of handwritten digits\n(http://yann.lecun.com/exdb/mnist/)\n\nAuthor: Aymeric Damien\nProject: https://github.com/aymericdamien/TensorFlow-Examples/\n'

In [2]:
import tensorflow as tf
import tensorflow as tf
from sklearn import metrics, cross_validation
import pandas as pd
import numpy as np
from numpy import genfromtxt

data = pd.read_csv('EURUSD1_new.csv', sep=',', names=['Time', 'Open', 'Max', 'Min', 'Close', 'Value'])
data_process = pd.read_csv('EURUSD1_new_pars_value.csv', sep=',', names=['GlobalOpen', 'Max1', 'Min1', 'Close1', 'Value1', 'Open2', 'Max2', 'Min2', 'Close2', 'Value2', 'Open3', 'Max3', 'Min3', 'Close3', 'Value3','Open4', 'Max4', 'Min4', 'Close4', 'Value4','Open5', 'Max5', 'Min5', 'GlobalClose', 'Value5'])

del data['Time']
data_array = data.values
data_array_upd = data_array.reshape(13000, 25)
data_array_upd.shape
X_data = pd.DataFrame(data_array_upd)


ii = 0
y = []
for i in data_process.GlobalOpen:
    if (data_process['GlobalClose'].values[ii] > i):
        y.append(1)
    else:
        y.append(0)
    ii+=1
    
def to_vector(y,clas =  2):
    num_labels = len(y)
    index_off = np.arange(num_labels)*2
    labels_one_hot = np.zeros((num_labels, clas))
    labels_one_hot.flat[index_off + y] = 1
    return labels_one_hot
y_one_hot = to_vector(y, 2)
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X_data.values, y_one_hot, test_size=0.2, random_state=42)
y_one_hot

array([[ 0.,  1.],
       [ 1.,  0.],
       [ 0.,  1.],
       ..., 
       [ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.]])

In [3]:
# Parameters
learning_rate = 0.001
training_iters = 200000
batch_size = 128
display_step = 10

# Network Parameters
n_input = 25 # MNIST data input (img shape: 28*28)
n_classes = 2 # MNIST total classes (0-9 digits)
dropout = 0.75 # Dropout, probability to keep units

# tf Graph input
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)

In [4]:
# Create some wrappers for simplicity
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool2d(x, k=1):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')


# Create model
def conv_net(x, weights, biases, dropout):
    # Reshape input picture
    x = tf.reshape(x, shape=[-1, 5, 5, 1])

    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2)

    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [5]:
# Store layers weight & bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = conv_net(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.initialize_all_variables()

ValueError: filter must not be larger than the input: Filter: [5x5] Input: [3x3]

In [ ]:
sess = tf.Session()
sess.run(init)
for i in range(100):
    average_cost = 0
    number_of_bathes = int(len(X_train) / batch_size)
    for start, end in zip(range(0, len(X_train), batch_size), range(batch_size, len(X_train), batch_size)):
        sess.run(optimizer, feed_dict={x: X_train[start:end], y: y_train[start:end], keep_prob: dropout})
        average_cost += sess.run(cost, feed_dict = {x: X_train[start:end], y: y_train[start:end]})
    print("Epoch:", '%04d' % (i+1), 'cost', "{:.9f}".format(average_cost))
print("Finished optimization!")